Pour se connecter à ce notebook 
Besoin de lancer  l'image singularity de : _Containers/shortcakelight.sif

https://github.com/rnakato/ShortCake/tree/master
Est conçu spécifiquement pour les analyses de données single cell

Pour lancer l'image singularity et pouvoir se connecter aux notebooks jupyter:
srun --pty bash
singularity exec 6_Containers/shortcakelight.sif jupyternotebook.sh

puis faire le lien ssh sur l'ordi avec cette commande  que l'on ecrit dans le terminal local: 
ssh -A -t -t vgoupille@genossh.genouest.org -L 8888:localhost:8888 ssh cl1n030 -L 8888:localhost:8888
modifier le port (numero et noeud si besoin)

si probleme de connection 

ls -a
rm -r .vscode-server
puis se remet sur vscode 
et refait la liaison avec ssh remote (et reinstaller les extensions...)

Library strategy: microSPLiT
The data preprocessing and alignment was performed using a modified SPLiT-seq pipeline (https://github.com/Alex-Rosenberg/split-seq-pipeline). Modifications: using STAR with the splicing isoform detection swiwwe usededt the hioutghoutest-scored multimapping re. We also keptwening a ctional count based on the number igninggood alignments, since bacterial genomes are known to contain overlapping CDSs.
Genome_build: ASM904v1.45 and ASM80076v1.37 from EnsemblBacteria
Supplementary_files_format_and_content: Cell by gene matrix (with cells above threshold 200 UMI/cell); Gene names; Cell annotations (barcode and well for heat shock data, OD for B.subtilis growth curve). For B. Subtilis growth curve, only mRNA are included in the cell-by-gene matrix.
Supplementary_files_format_and_content: For the heat-shock data, barcodes in wells 1-24 belong to heat-shocked cells and in wells 25-48 belong to control cells.
 	

Ce texte décrit les stratégies et les méthodes utilisées pour traiter et aligner les données issues de l’expérience scRNA-seq réalisée avec la méthode microSPLiT.
	1.	Stratégie de séquençage (Library strategy) :
	•	La méthode utilisée est microSPLiT, qui est une adaptation de la technique SPLiT-seq, permettant de réaliser du scRNA-seq sans nécessiter d’isolation physique des cellules.
	2.	Prétraitement et alignement des données :
	•	L’alignement des séquences a été effectué avec une version modifiée du pipeline SPLiT-seq disponible sur GitHub (https://github.com/Alex-Rosenberg/split-seq-pipeline).
	•	Modifications apportées :
	•	Utilisation de STAR (un logiciel d’alignement des lectures RNA-seq) avec une détection des isoformes d’épissage.
	•	Sélection des meilleurs alignements multimapping selon un critère de score élevé.
	•	Conservation d’un compte fonctionnel basé sur le nombre d’alignements de bonne qualité, ce qui est pertinent pour les génomes bactériens où plusieurs CDSs (séquences codantes) peuvent se chevaucher.
	3.	Références génomiques utilisées :
	•	Les génomes de référence utilisés pour l’alignement sont :
	•	ASM904v1.45
	•	ASM80076v1.37
	•	Ces versions proviennent de la base de données EnsemblBacteria.
	4.	Fichiers supplémentaires et leur contenu :
	•	Les fichiers supplémentaires fournis incluent :
	•	Matrice cellule-gène (contenant uniquement les cellules ayant un minimum de 200 UMI/cellule).
	•	Noms des gènes.
	•	Annotations des cellules :
	•	Pour les données du stress thermique (heat-shock) : les cellules sont annotées par code-barres et puits.
	•	Pour la courbe de croissance de Bacillus subtilis : les cellules sont annotées selon leur densité optique (OD). Seuls les ARNm (mRNA) sont inclus dans la matrice cellule-gène.
	5.	Organisation des cellules dans l’expérience de stress thermique :
	•	Les cellules sont disposées dans 48 puits, avec une distinction entre conditions :
	•	Puits 1-24 : cellules soumises au stress thermique.
	•	Puits 25-48 : cellules témoins (non soumises au stress thermique).

replica 1 : M11 => B. subtilis PY79 (OD0.5-OD3.2)

replica 2 : M14 => B. subtilis PY79 (OD0.5-OD6.0)

other plate : M15 =>	B. subtilis PY79 + E. coli MW1255

# info sur les objets de type Anndata
https://anndata.readthedocs.io/en/latest/tutorials/notebooks/getting-started.html

# info sur l'utilisation de Scanpy 
https://scanpy.readthedocs.io/en/stable/tutorials/basics/clustering.html

In [1]:
import pandas as pd
import scanpy as sc
import numpy as np
from scipy.sparse import csr_matrix

# Charger les fichiers CSV
barcodes = pd.read_csv("7_Article/data/GSE_extracted/GSM4594094_M11_barcodes.csv", header=None)[0]  # Liste des cellules
genes = pd.read_csv("7_Article/data/GSE_extracted/GSM4594094_M11_genes.csv", header=None)[0]  # Liste des gènes
dcm = pd.read_csv("7_Article/data/GSE_extracted/GSM4594094_M11_dcm.csv", header=None)  # Matrice d'expression (comptages)


#Preview des données
print(barcodes.head())
print(genes.head())
print(dcm.head())

# Vérifier les dimensions
print(f"Barcodes: {len(barcodes)}, Genes: {len(genes)}, DCM shape: {dcm.shape}")

0    AAACATCGAACAACCA_41_20200401_M11_0510_K12_168
1            AAACATCGAACTCACC_12_20200330_M11_0414
2     AAACATCGAAGACGGA_1_20200401_M11_0510_K12_168
3            AAACATCGAAGAGATC_35_20200330_M11_0414
4     AAACATCGAAGGTACA_6_20200401_M11_0510_K12_168
Name: 0, dtype: object
0    BSU_rRNA_29_BSUBT
1     BSU_rRNA_5_BSUBT
2     BSU_rRNA_2_BSUBT
3    BSU_rRNA_20_BSUBT
4    BSU_rRNA_12_BSUBT
Name: 0, dtype: object
        0          1           2           3           4           5     \
0   0.100000    0.10000    0.100000    0.100000    0.100000    0.100000   
1  95.029745  107.04071  107.829599  104.141107  108.596266  110.426821   
2   0.411111    0.30000    0.411111    0.411111    0.411111    0.411111   
3   0.000000    0.00000    0.000000    0.000000    0.000000    0.000000   
4   0.225000    0.22500    0.225000    0.100000    0.225000    0.225000   

         6           7           8          9     ...  7558  7559  7560  7561  \
0    0.100000    0.100000    0.100000   0.100000  ..